In [3]:
import numpy as np
from scipy.stats import weibull_min
from scipy.special import gamma # NOT scipy.stats.gamma!

np.random.seed(43)
X = weibull_min.rvs(c=1, size=40, random_state=42)
X = np.sort(X)

scale = X.sum() / X.size # выборочное среднее
scale /= gamma(2)
scale

0.8889780968730877

In [4]:
c, loc, scale = weibull_min.fit(X)
print(c, loc, scale)

0.8466173812734985 0.02079930799913862 0.848644015205604


In [5]:
c, loc,scale = weibull_min.fit(X, floc=0, fc=1)
print(c, loc, scale)

1 0 0.8889631984743804
